# Shape of Galaxy Cluster from Galaxy Catalog

#### Matching galaxies onto clusters.
Extract positional information from galaxies and match them to same host cluster. 
Need to convert RA, DEC, Z or whatever coordinates into X,Y,Z

#### Axes finding algorithm
Use simplied inertia tensor
Eigenvectors are axes
Eigenvalues take square root are length per unit mass (how to divide by mass?) Total mass of all galaxies?
To simplify, take mass of all galaxies to be 1, so that not weighted by mass

## Matching galaxies to clusters

In [1]:
import numpy as np

# import pyplot and set some parameters to make plots prettier
import matplotlib.pyplot as plt
from code.plot_utils import plot_pretty
plot_pretty()

# setup notebook for inline figures
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D

In [2]:
from code.setup.setup import data_home_dir
import pyfits
#Read http://pythonhosted.org/pyfits/

datadir = data_home_dir()

#RedMapper Galaxy cluster data
redM_data = datadir+'buzzard-0_1.6_y3_run_redmapper_v6.4.20_lgt20_vl02_catalog.fit'
redM_list = pyfits.open(redM_data)
redM_data = redM_list[1].data
print "Number of clusters is ", len(redM_data)

#Galaxy File
gal_data = datadir+'buzzard-0_1.6_y3_run_redmapper_v6.4.20_lgt20_vl02_catalog_members.fit'
gal_list = pyfits.open(gal_data)
gal_data = gal_list[1].data

#test the number of unique clusters on galaxies
gal_match_cl = gal_data['MEM_MATCH_ID']
a = np.unique(gal_match_cl)
print 'Number of host clusters galaxies matched onto is ', len(a)
print "Number of galaxis is ", len(gal_data)

Number of clusters is  24243
Number of host clusters galaxies matched onto is  24243
Number of galaxis is  1564350


In [3]:
#Convert from (RA, DEC, Z) to (X, Y, Z) in order to build Cartesian inertial tensor. 
#Add coordinates onto the existing gal_dat and redM_data records array
from astropy.cosmology import FlatLambdaCDM #as cosmo #H0 = 67.7
from astropy.coordinates import SkyCoord
from astropy import units as u
import pickle

#Define cosmology
h = 0.7
cosmo = FlatLambdaCDM(H0=h*100, Om0=0.3, Tcmb0=2.725)

#Convert coordinates
#vec = hp.ang2vec(angle[0],angle[1])
gal_RA = np.copy(gal_data['RA']); gal_DEC = np.copy(gal_data['DEC']); gal_Z = np.copy(gal_data['ZSPEC'])
gal_R = np.copy(gal_data['R']); gal_match_ID = gal_data['MEM_MATCH_ID']
redM_RA = np.copy(redM_data['RA']); redM_DEC = np.copy(redM_data['DEC']); 
redM_Z = np.copy(redM_data['Z']); redM_ID=redM_data['MEM_MATCH_ID']

redM_comv = cosmo.comoving_distance(redM_Z)

In [21]:
#Array that matches galaxies with clusters. Each cluster entry should have rx, ry, rz for each galaxy.

#Find matching galaxies to a cluster
#Change from RA, DEC, Z to rx, ry, rz. Need to figure out the line of sight direction. 
#Write a function for the shape tensor. 

#For Galaxies selection. Using DM particles. No longer needed.

"""
gal_sel = (gal_match_ID == redM_ID[0])
gal_data_sel = gal_data[gal_sel]
gal_Z_sel = gal_Z[gal_sel]
num_mem_gal = len(gal_data_sel)

#build coordinate system
centr = SkyCoord(redM_RA[0]*u.degree, redM_DEC[0]*u.degree, distance=redM_comv[0]*u.Mpc, frame='icrs')
centr.representation = 'cartesian'
gal_comv = cosmo.comoving_distance(gal_data_sel['Z']) #!Should use ZSPEC for true distance, but use Z for projected assuming same as cluster center
gal_coord = SkyCoord(gal_data_sel['RA']*u.degree, gal_data_sel['DEC']*u.degree, distance=gal_comv*u.Mpc, frame='icrs')
gal_coord.representation = 'cartesian'

#Converting astopy coordinates to number array
gal_coord_x = gal_coord.x.value; gal_coord_y = gal_coord.y.value; gal_coord_z = gal_coord.z.value
centr_x = centr.x.value; centr_y = centr.y.value; centr_z = centr.z.value
"""
print "Check coordinates"
#Check if SkyCoord icrs frame has same X,Y,Z as hp.vec2ang

Check coordinates


In [25]:
"""
Updated April 16, 2018
Follow convention of Ken Osato: Use reduced quadropole moment to find axis ratio of ellipsoidal cluster
1. Project onto principle axes spitted out by quadropole tensor
2. Do not remove particles. Particles chosen for those inside Rvir
3. Use Reduced tensor
4. q, s refer to ratio of minor to major, and intermediate to major axis

Returns:
converge -- Boolean
[a,b,c] -- normalized major, intermediate, minor axes lengths (only ratio matters in reduced tensor)
[lx, ly, lz] -- direction of minor, intermediate, major in original (non-rotated) basis
"""

def quad_moment(ptcl_coord, centr, rvir):
    centr_x = centr[0]; centr_y = centr[1]; centr_z = centr[2]
    ptcl_coord_x = ptcl_coord[0]; ptcl_coord_y = ptcl_coord[1]; ptcl_coord_z = ptcl_coord[2]

    rx = ptcl_coord_x - centr_x; ry = ptcl_coord_y - centr_y; rz = ptcl_coord_z - centr_z 

    R_range = np.sqrt(rx**2. + ry**2. + rz**2.)
    #rmax = np.sqrt(np.max(r_mem_ptcl[:,3]))
    #print "Number of particles before selection is ", len(rx)
    
    #Choose particles inside Rvir
    ptcl_range = np.where(R_range < rvir)
    rx = rx[ptcl_range]; ry = ry[ptcl_range]; rz = rz[ptcl_range]
     
    num_mem_ptcl = len(rx)
    #print "Number of particles inside virial radius is ", num_mem_ptcl

    #Building quadrupole tensor. 
    Rp = np.sqrt(rx**2. + ry**2. + rz**2.)
    r = np.matrix([rx,ry,rz])
    r_rdu = r/Rp
    M_rdu = r_rdu*r_rdu.T #Initial quadrupole tensor before iteration

    #Finding eigvec, eigval
    M_eigval, M_eigvec = np.linalg.eig(M_rdu)
    sort_eigval = np.argsort(M_eigval)[::-1]
    a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl) #a, b, c major, intermediate, minor
    lx, ly, lz = M_eigvec.T[sort_eigval][::-1] #lx, ly, lz minor, intermediate, major (order reversed from a, b, c)
    lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]
    
    #Sanity check
    """
    print "r_rdu", r_rdu
    check_eig = M_rdu.dot(lx) - num_mem_ptcl*c**2.*lx
    print "M_rdu.dot(lx) ", np.dot(np.array(M_rdu), lx)
    print "check_eig ", check_eig
    print "lx is ", lx
    print "M_eigvec.T[sort_eigval], ", M_eigvec.T[sort_eigval]
    print "M_eigvec[:,0] ", M_eigvec[:,0]
    print "M_eigvec[sort_eigval] ", M_eigvec[sort_eigval]
    print "M_eigvec", M_eigvec
    print "sort_eigval ", sort_eigval
    """
    
    #Initial conditions
    q_prev = 1.; s_prev = 1.
    converge = False
    conv_iter = 0

    P_tot = np.eye(3) #the multiplicative product of all projections done over each iteration
    while (not converge) & (conv_iter < 100):
        #Change of basis
        P_axis = np.matrix([lx,ly,lz])
        P_tot = P_axis*P_tot
        r_proj = P_axis*r
        rx = np.array(r_proj[0,:])[0]; ry = np.array(r_proj[1,:])[0]; rz = np.array(r_proj[2,:])[0]

        #New iteration
        q_cur = c/a; s_cur = b/a #Osato conventaion
        Rp = np.sqrt((rx/q_cur)**2. + (ry/s_cur)**2. + rz**2.)
        r = np.matrix([rx, ry, rz])
        r_rdu = r/Rp
        M_rdu = r_rdu*r_rdu.T
        M_eigval, M_eigvec = np.linalg.eig(M_rdu)
        sort_eigval = np.argsort(M_eigval)[::-1]
        a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl)
        lx, ly, lz = M_eigvec.T[sort_eigval][::-1]
        lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]
        
        #test converge
        conv_err = 1e-6
        conv_s = np.abs(1 - s_cur/s_prev); conv_q = np.abs(1 - q_cur/q_prev)
        converge = (conv_s < conv_err) & (conv_q < conv_err)
        #print "Conv_s, conv_q ", conv_s, conv_q
        #print "Number of particles ", len(rx)
        #print "a, b, c ", a, b, c
        #print "q, s are ", q_cur, s_cur  
        #print "lx", lx
        #print 'converge is ', converge
        #print '\n'
        conv_iter += 1
        q_prev = q_cur; s_prev = s_cur
    
    #find lx, ly, lz in original basis
    P_inv = np.linalg.inv(P_tot)
    l_new_basis = np.matrix([lx,ly,lz]).T
    l_orig_basis = np.transpose(P_inv*l_new_basis)
    lx_orig = l_orig_basis[0]; ly_orig = l_orig_basis[1]; lz_orig = l_orig_basis[2]
    
    
    return converge, [a,b,c], [lx_orig, ly_orig, lz_orig]

In [26]:
from code.Heidi_read_halo_particles import read_halo_ptcl
#For testing the quad_moment function

ra = 64.3599024; dec = 16.68787569;
x_cen = 273.26001; y_cen = 569.31482; z_cen = 189.31299
red_cen = 0.23191588
chi_cen = np.sqrt(x_cen**2 + y_cen**2 + z_cen**2)
rbin = int(chi_cen//25)
rvir = 2.549908


x, y, z = read_halo_ptcl(ra, dec, red_cen, x_cen, y_cen, z_cen, Rmax=rvir)
ptcl_coord = [x,y,z]; centr = [x_cen, y_cen, z_cen]
#print ptcl_coord, centr

converge, axes_len, axes_dir = quad_moment(ptcl_coord, centr, rvir)
print converge, axes_len, axes_dir
    

True [0.5773504279935158, 0.31355738012861206, 0.23943732103822274] [matrix([[ 0.73342967,  0.67975759, -0.00324522]]), matrix([[ 0.00301255, -0.00802432, -0.99996327]]), matrix([[ 0.67975866, -0.73339295,  0.00793308]])]


[[0 0 1]
 [0 0 1]
 [0 0 1]]
1
